In [33]:
import re
import os
from datasets import load_dataset
file_path = "data/data.txt"

In [34]:
if not os.path.exists(file_path):
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
else:
    # 2. 데이터 로드 (split='train'을 명시해야 Dataset 객체로 바로 반환됩니다)
    # 기존 캐시 문제를 방지하기 위해 download_mode를 추가할 수 있습니다.
    dataset = load_dataset("text", data_files={"train": file_path}, split='train')

In [35]:
def parse_line(example):
        line = example['text']
        
        # [Regex] 앞부분 숫자 6개 (아이템 개수 및 가치)
        input_match = re.match(r'^(\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+)', line)
        input_vals = input_match.group(1) if input_match else ""
        
        # [Regex] 대화 내용 (<selection> 이전까지)
        dialogue_match = re.search(r'(?:YOU:|THEM:).*?(?=<selection>)', line)
        dialogue = dialogue_match.group(0).strip() if dialogue_match else ""
        
        # [Regex] 보상(reward) 숫자 추출
        reward_match = re.search(r'reward=(\d+)', line)
        reward = int(reward_match.group(1)) if reward_match else 0
        
        # [Regex] 합의 여부 (agree / disagree)
        is_agree = "agree" in line and "disagree" not in line
        
        return {
            "input": input_vals,
            "dialogue": dialogue,
            "reward": reward,
            "is_agree": is_agree
        }

In [36]:
parsed_ds = dataset.map(parse_line)
    
# 데이터가 정상 로드되었는지 확인
print(f"✅ 총 로드된 데이터: {len(parsed_ds)}개")
if len(parsed_ds) > 0:
    print("첫 번째 데이터 샘플:", parsed_ds[0])

Map: 100%|██████████| 12234/12234 [00:00<00:00, 50322.52 examples/s]

✅ 총 로드된 데이터: 12234개
첫 번째 데이터 샘플: {'text': '1 0 4 2 1 2 YOU: i would like 4 hats and you can have the rest . <eos> THEM: deal <eos> YOU: <selection> item0=0 item1=4 item2=0 <eos> reward=8 agree 1 4 4 1 1 2', 'input': '1 0 4 2 1 2', 'dialogue': 'YOU: i would like 4 hats and you can have the rest . <eos> THEM: deal <eos> YOU:', 'reward': 8, 'is_agree': True}


In [50]:
best_df = parsed_ds.filter(lambda x: x['is_agree'] and x['reward'] == 10).to_pandas()

Filter: 100%|██████████| 12234/12234 [00:00<00:00, 219726.27 examples/s]


In [51]:
# 2. 대화의 질을 평가하는 함수 정의
def evaluate_quality(dialogue):
    turns = dialogue.count("YOU:") + dialogue.count("THEM:")
    length = len(dialogue)
    # 3~6턴 사이이면서, 글자 수가 100~250자 사이인 것
    if 3 <= turns <= 6 and 100 <= length <= 250:
        return True
    return False

In [52]:
filtered_candidates = best_df[best_df['dialogue'].apply(evaluate_quality)]

In [53]:
print(f"후보군 개수: {len(filtered_candidates)}")
for i, row in filtered_candidates.head(10).iterrows():
    print(f"--- 후보 {i} (점수: {row['reward']}, 길이: {len(row['dialogue'])}) ---")
    print(row['dialogue'])
    print("\n")

후보군 개수: 878
--- 후보 0 (점수: 10, 길이: 221) ---
YOU: hi i would like the book and ball and you can have the hats <eos> THEM: i can give you either the book or the ball <eos> YOU: ill take the book <eos> THEM: ok i will take the hats and ball <eos> YOU: deal <eos> THEM:


--- 후보 1 (점수: 10, 길이: 148) ---
YOU: lemme get that book and ball ! <eos> THEM: i need 2 balls and a hat <eos> YOU: i'll take the book and two hats then ? <eos> THEM: ok <eos> YOU:


--- 후보 10 (점수: 10, 길이: 149) ---
THEM: if i can have the balls and the book you can have the hats <eos> YOU: sounds good <eos> THEM: great ! thank you ! <eos> YOU: thanks <eos> THEM:


--- 후보 13 (점수: 10, 길이: 206) ---
YOU: i'd like the hats and books <eos> THEM: ill give you all books with a hat , i get to keep a hats <eos> YOU: how about this , i get the hats you take everything else . deal ? <eos> THEM: deal <eos> YOU:


--- 후보 14 (점수: 10, 길이: 187) ---
YOU: hi , i would like 2 balls and 1 hat . <eos> THEM: i'd like book and 1 ball or hat ? <eos>